In [1]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline


#theano imports
import lasagne
import theano
import theano.tensor as T
import sys
sys.setrecursionlimit(100000)
floatX = theano.config.floatX

## Про библиотеку

Для этого семинара вам понадобится библиотека AgentNet

В предположении, что вы уже имеете Bleeding Edge версию theano и lasagne, доставить её на Linux и Mac OS можно вот как:

```
pip install --upgrade https://github.com/yandexdataschool/AgentNet/archive/3e32be8c348f93a84541cc1c52f398cbdf05ec3c.zip
```
** Внимание: текущий семинар не совместим с python2 **

В предположении, что python - это тот питон, которым вы пользуетесь в тетрадке.
Если у вас в тетрадке используется другой питон - используйте его, например
* если вы используете Py3, а по умолчанию стоит Py2 - `python3 setup.py install`
* `sudo jupyter notebook` -> `sudo python setup.py install`
* если вдруг возникают проблемы, которые не решаются комбинацией sudo и python2/python3 - пишите, мы вас вылечим :)


## Небольшой варнунг

Задача, которая решается в этом семинаре - довольно простая, а сети - ничтожно маленькие. 

Вы, конечно, можете использовать GPU, но не факт, что она даст вам выигрыш в рантайме, а вот при компиляции от неё можно точно ожидать замедления процесса раза в 2.

In [ ]:
%load_ext autoreload
%autoreload 2

# Stack-augmented RNN
![caption](https://usercontent1.hubstatic.com/6172838_f260.jpg)



## Сегодня в меню

__Задача__ - научить нейронку порождать последовательности вида `|`$ a^n b^m c^{n+m} $
 * n и m - целые числа в определённом интервале
 * Чего хотим:
  * Чтобы последовательность имела правильную форму - `|`, сколько-то a, сколько-то b, сколько-то c
     * ||aaacbbcba - не ок
  * Чтобы количество букв C было как можно ближе к сумме количеств букв a и b
     * в идеале, чтобы оно совпадало
     
     
Что попробуем:
 * Ванильный RNN
 * Stack-augmented RNN
 
Идейно мы будем учить их в режиме Language Model (см. предыдущий семинар) - читать последовательность и предсказывать следующий символ

### Но для начала сделаем генератор "правильных" последовательностей

In [ ]:
def generate_sequence(batch_size = 10,crop_length = 100 ):
    """
    Generates sequence from pattern [0  1*n 2*m 3*(n+m)]
    """
    sequences=[]
    for i in range(batch_size):
        seq = [0]
        
        
        #заполните строку seq последовательностями длины ровно crop_length
        #из подстрок вида [0  1*n 2*m 3*(n+m)],
        # где n,m - случайные числа от 1 до 15 включительно
        #примерs правильных строк есть на 2 клетки ниже после "ожидаемый результат"
        
        <ваш код тут>

        
        
        assert len(seq) == crop_length
        
        sequences.append(seq)
    return np.array(sequences,dtype='int32')

alphabet = np.array(list('|abc'))

### Тестики

In [ ]:
%%time
map(''.join,map(alphabet.__getitem__,generate_sequence(25,100)))

__Ожидаемый результат__ табы сверху - список из строчек вида


`
 ...
 '||aaaaaaaaabbbcccccccccccc|aaaabccccc|aaaaaaaaabbccccccccccc|aaaaaaaaabbbbccccccccccccc|aaaaaaaaaabb',
 '||aaaaaaaaaaaabbbbbbbbbbcccccccccccccccccccccc|aaaabbbbbbbbbbbbcccccccccccccccc|aaaabbbbbbbbcccccccc',
 '||aaaaaaabbbbbcccccccccccc|aaaaabbbbbbccccccccccc|aaaaaaaaaaaabbbbbccccccccccccccccc|abbbbbbbbbbbccc',
 '||aaaaaaaaaaaabbbbcccccccccccccccc|aaaaaaaabbbbbbbbcccccccccccccccc|aaaaaaaaaaaabbbbbbbbcccccccccccc',
 '||aaaaaaaaaaaaabcccccccccccccc|aaaaaaaaaaabbbbbbbbbbbbccccccccccccccccccccccc|aaaaaaaaaaaaaabbbbbbbb',
 '||abbbbccccc|aaaaabbbbbbbbbbbbccccccccccccccccc|aaaaaaaaaaaaabbbbbbbbbbbbccccccccccccccccccccccccc|a',
 '||aaaaaaaabbbbcccccccccccc|aaaabbbbbbbbbccccccccccccc|aaaaaaaaaaaabbbbbbbbbbbccccccccccccccccccccccc',
 '||aaaaaaaabbbbbbbbbbbbbccccccccccccccccccccc|aaaaaabbbbbbbbbbbbbbcccccccccccccccccccc|aaaaaaaaaaaaab',
 ...
`

In [ ]:
from metrics import get_metrics

In [ ]:
#формальные тестики
gen_sequences = generate_sequence(1000,500)
correctness_ratio, c_count_mae = get_metrics(gen_sequences,alphabet)

# Все порожлённые последовательности должны иметь правильный формат - | a+ b+ c+
assert correctness_ratio == 1.0
# Во всех последовательносях количество букв C должно равняться сумме количеств букв a и b
assert c_count_mae == 0

#и выборка должна иметь правильный размер
assert len(gen_sequences) == 1000
assert len(gen_sequences[0]) == 500

# Константы и глобальные переменные

* Просто несколько чиселок, которые будут использоваться далее по коду

In [ ]:
# Длина генерируемых последовательностей
SEQ_LENGTH = 100

# сколько последовательностей в одном минибатче
BATCH_SIZE = 100

# сколько итераций всего нужно провести
N_EPOCHS = 5000

#раз в какое количество эпох нужно печатать качество нейронки
REPORT_RATE = 50

In [ ]:
#Последовательность букв на вход
sequences_batch = T.matrix(dtype="int32",name="reference_sequences")

#Её размер (theano-выражение)
batch_size = sequences_batch.shape[0]


# Поучим RNN

Структурно нам нужно создать вот такую махину, которая будет описывать 1 шаг RNN во времени

* время - слева направо
* входы - снизу, выходы - сверху

![scheme](./rnn.png)

где 
* prev rnn state - предыдущее состояние RNN
* input letter - буква на входе
* next rnn state - следующее состояние RNN
* generate_letter - выбор одной конкретной следующей буквы пропорционально предсказанным вероятностям
* всё остальное - слои лазаньи

In [1]:
import lasagne
from lasagne.layers import DenseLayer, ElemwiseSumLayer, InputLayer, EmbeddingLayer, NonlinearityLayer
import agentnet
from agentnet.resolver import ProbablisticResolver
from agentnet.agent import Generator

In [ ]:


#observation = input letter - сюда на каждом шаге подаётся очередная буква из входной последовательности
output_shape = (None,)

observation_layer = InputLayer(output_shape,name="obs_input")


# embedding
n_tokens = len(alphabet)
obs_embedding = EmbeddingLayer(observation_layer,
                               input_size=n_tokens,
                               output_size=n_tokens,
                               name = "input_embedding")



# RNN memory

#число нейронов в скрытом слое
n_hid_1 = 70

#Сюда прицепится предыдущее состояние RNN
#~ prev rnn state
prev_rnn_layer = InputLayer((None,n_hid_1),name="prev_rnn_state")


#Построим RNN вручную - по схеме выше

rnn_frominput = полносвязный слой без нелинейности размером n_hid_1, растёт из embedding-а

rnn_fromhidden = полносвязный слой без нелинейности размером n_hid_1, растёт из prev rnn layer-а

#если не хотите учить RNN с двумя наборами b-шек, скажите одному из входов b=None


rnn_sum = поэлементная сумма их (ElemwiseSumLayer)

rnn = нелинейность (по умолчанию лучше взять tanh)



# В этом словаре лежат пары  {выход скрытого состояния: вход скрытого состояния с предыдущего шага}

from collections import OrderedDict
memory_dict = OrderedDict([
            (rnn,prev_rnn_layer),
    ])


#Вероятности букв

probability_layer = DenseLayer(rnn,
                               num_units = n_tokens,
                               nonlinearity=  lasagne.nonlinearities.softmax,
                               name="policy_original")

#resolver - выбирает конкретную букву пропорционально вероятностям

resolver = ProbablisticResolver(probability_layer,
                                assume_normalized=True,
                                name="resolver")


#Проверка, совпадают ли формы у состояний. Должна пройти, если вы не сделали чего-то ужасного
assert tuple(lasagne.layers.get_output_shape(resolver)) == tuple(output_shape)






#Создаём рекуррентный генератор, который
# - имеет 1 вход - observation layer
# - имеет память, описанную в memory dict ( один слой RNN )
# - генерирует буквы с вероятностями, полученными на probability layer
# - принимает решение, какую букву генерить, на слое resolver layer - пропорционально вероятностям

agent = Generator(
    observation_layer,
    memory_dict,
    probability_layer,
    resolver
    )


# А теперь покрутим его

* Применим агента к последовательности букв на входе по схеме выше
* Получим тот же самый формат, что и у lasagne.layers.RecurrentLayer, GRU и LSTM

In [ ]:
sessions = agent.get_sessions(session_length=SEQ_LENGTH,
                             recorded_sequences=sequences_batch,
                             batch_size=batch_size,)


# состояние RNN - сгенерирнованные буквы  - вероятности букв
agent_states,          action_seq,           probas_seq       =  sessions


# Вытаскиваем состояние конкретного RNN
rnn_seq = agent_states[rnn]


# Из всего этого нам нужны только вероятности :)
# probas_seq


### А дальше всё как в обычной лазанье

### Веса

In [ ]:
#Получим веса
weights = lasagne.layers.get_all_params(resolver,trainable=True)
weights

In [ ]:
total_weights = int(T.sum([T.prod(w.shape) for w in weights]).eval())
print "Всего весов:", total_weights

#если вы экспериментируете с размером нейронки - удалите следующую строку
assert  5200 < total_weights <= 5700

# Функция потерь

На этот раз для ускорения обучения мы предсказываем не 1 последний символ, а предсказываем следующий символ в каждый момент

In [ ]:

# Возьмём все предсказания кроме последнего(для него мы не знаем "следующего")
predicted_probas = probas_seq[:,:-1]

# введём минимальную вероятность - чтобы не получить -Inf в логарифме
predicted_probas = T.maximum(predicted_probas,1e-10)

# Правильные ответы - для 0-го выхода - первый вход, для 1-го - второй вход и так далее
# нулевой вход нам не нужен
references = sequences_batch[:,1:]



# Обычная кроссэнтропия
model_loss = lasagne.objectives.categorical_crossentropy(
    predicted_probas.reshape([-1,n_tokens]),
    references.ravel()
).mean()





In [ ]:
#Регуляризатор для вкуса
from lasagne.regularization import regularize_network_params, l2
reg_l2 = regularize_network_params(resolver,l2)*10**-5
loss = model_loss + reg_l2


In [ ]:
updates = ваш любивый оптимизатор

# Скомпилируем это всё.
* В первый раз, если SEQ_LENGTH больше 25, можно успеть выпить кофе;
* Если вы на ещё и на GPU - съесть тортик.
* К слову, у него тоже есть слои

![canvas](http://www.rabstol.net/uploads/gallery/main/322/rabstol_net_cakes_30.jpg)

* p.s Cake is a lie!

In [ ]:
train_fun = theano.function([sequences_batch],[loss],updates=updates)
evaluation_fun = theano.function([sequences_batch],[loss,model_loss,reg_l2])

# Генерация новых символов

In [ ]:
# сюда мы подадим размер батча для генерации
gen_batch_size = T.scalar('generated batch size','int32')


# аналогично предыдущему, но теперь мы не даём ему "правильную" последовательность, 
# и он кушает свои собственные выходы с предыдущего момента времени
_,generated_action_seq,_ = agent.get_sessions(session_length=SEQ_LENGTH,
                             batch_size=gen_batch_size,)


In [ ]:
#допиваем кофе
get_sequences = theano.function([gen_batch_size],generated_action_seq)

In [ ]:
#Пока наша сетка ничено не знает (как Jon Snow, только не <spoiler>)
map(alphabet.__getitem__,get_sequences(3))

# Цикл обучения

* Всё как обычно - учимся и раз в 100 итераций считаем метрики

* Метрики нас интересуют 3 - 
 * llh - простой loglikelihood - должен падать +- шум
 * доля правильных последовательностей - с какой вероятностью rnn сгенерит подряд сколько-то a, потом b, потом c - и потом начнёт новую строку.
   * для формалистов - доля строк, которые полностью матчатся регээкспом "^|a+b+c+"
   * тоже должен в целом расти, пока не выйдёт на асимптоту близь единицы
 * ошибка в количестве C
   * рассматриваем только "правильные" последовательности по определению из предыдущего пункта
   * считаем количество букв a (n), количество букв b (m). Правильное количество букв с - n+m
   * Считаем средний модуль разности между сгенерированным количеством С и правильным.
   * Формально метрика называется mean absolute error или MAE

In [ ]:
from collections import defaultdict
metrics = defaultdict(dict)

In [ ]:

for i in range(N_EPOCHS):
    
    #породим строки
    new_batch = generate_sequence(BATCH_SIZE,SEQ_LENGTH)
    #Потренируем чудовище
    train_fun(new_batch)
    
    #Once in a while, напечатаем метрики
    if i % REPORT_RATE==0:
        
        loss_components = evaluation_fun(new_batch)
        print "iter:%i\tfull:%.5f\tllh:%.5f\treg:%.5f"%tuple([i]+map(float,loss_components))        
        
        metrics['crossentropy'][i]=float(loss_components[1])
        

        examples = get_sequences(1000)
        
        correctness_ratio,c_count_mae = get_metrics(examples,alphabet)
        

        metrics["correctness_rates"][i] = correctness_ratio
        metrics["c_count_errors"][i]=c_count_mae
        
        print "Доля последовательностей правильной формы: %.5f"%(correctness_ratio)
        print "MAE Ошибка по количеству C среди правильных: %.5f"%(c_count_mae)
        
        for tid_line in examples[:3]:
            print ' '.join(map(alphabet.__getitem__,tid_line))

    

In [ ]:
for metric in metrics:
    plt.figure(figsize=[14,8])
    plt.plot(*zip(*sorted(metrics[metric].items(),key=lambda (k,v):k)),label=metric)
    plt.legend(loc='best')
    plt.ylabel("popugai")
    plt.xlabel("epoch")
    plt.grid()

## тестики

In [ ]:


get_5_last = lambda metric_name: map(lambda v: v[1],sorted(metrics[metric_name].items(),key=lambda v:v[0])[-5:])


#проверяем, что минимум по последним 5 значениям метрики не хуже порога

assert min(get_5_last("crossentropy")) <= 0.3
assert min(get_5_last("c_count_errors")) <= 3
assert min(get_5_last("correctness_rates")) >= 0.8


In [ ]:
#запомним их в отдельную переменную
rnn_metrics = metrics

# Stack RNN

Теперь заведём Stack-augmented RNN.

Оригинальная статья - http://arxiv.org/abs/1503.01007
(там есть картинки)

Идея - давайте мы выделим нашей RNN-ке стэк, которым она сможет управлять при помощи трёх операций

* push - сдвиг стэка на единицу вглубь, добавление входного элемента
* pop - сдвиг всех элементов на единицу ближе к выходу
* no-op - сохранение состояния стэка

При этом все эти операции обобщены так, чтобы их можно было выполнять с коэффициентами от 0 до 1
* 0 - операция не выполняется
* 1 - операция выполняется в полной мере
* что-то между - операция выполняется отчасти
* При этом  
  * `0 <= push, pop, no-op <= 1`
  * `push + pop + no-op = 1`

В итоге обновление стэка выглядит так

```Stack(depth i, t+1) = push * Stack(depth i+1, t) + pop * Stack(depth i-1, t) + no-op * Stack(depth i, t)```


* В качестве "вводимого" элемента для push можно использовать элемент, полученный из скрытого состояния сети
* При выполнении pop, элемент стэка на максимальной глубине заполняется нулевыми значениями с соответствующим коэффициентом
* Самое первое (depth 0) значение в стэке влияет на новое значение рекуррентного слоя



Как это всё делать:
 * Создадим слой, реализующий стэковую память
 * Воткнём его в сеть
 * ???
 * PROFIT!!!



# Стэковая память

In [ ]:
from lasagne.layers.base import MergeLayer

class StackAugmentation(MergeLayer):
    def __init__(self,
                 observation_input,
                 prev_state_input,
                 controls_layer,
                 **kwargs):

        
        #default name
        if "name" not in kwargs:
            kwargs["name"] = "YetAnother"+self.__class__.__name__
        
               
        super(StackAugmentation, self).__init__([observation_input,prev_state_input,controls_layer], **kwargs)
        
            



    def get_output_for(self, inputs, **kwargs):
        """
            Нам на вход приходят
             - вводимый элемент формы [None, ширина стэка]
             - состояние стэка на предыдущем шаге, форма [None,глубина стэка, ширина стэка]
             - вектор управления стэком, форма [None, 3] - push, pop и no-op соответственно
             
            На выход мы хотим новое состояние стэка
        """
        
        
        #кто они
        input_val,prev_stack,controls = inputs
        assert input_val.ndim==2
        
        
        #немного преобразований формы (чтобы вам не нужно было делать это самосттоятельно)
        controls = controls.reshape([-1,3,1,1])    
        input_val = input_val[:,None,:]
        zeros_at_the_top = T.zeros_like(prev_stack[:,0,None,:])
        
        # операции управления стэком
        a_push,a_pop,a_no_op = controls[:,0],controls[:,1],controls[:,2]
        
        
        # Промежуточный этап - подготовим версии стэка, сдвинутые на единицу вверх или вниз.
        # Дописывать крайние элементы проще всего через T.concatenate(axis=1) или T.horizontal_stack
        
        
        
        stack_popped = стэк, сдвинутый вниз на 1 единицу глубины(начало выброшено), в конец которого дописан zeros_at_the_top
        
        
        stack_pushed = стэк, сдвинутый вверх на 1 единицу глубины (конец выброшен), в начало которого дописан input_val
        
        
        new_stack = формула для нового стэка с использованием 3 сигналов управления и соответствующих вариаций стэка

        return new_stack
    
    
    
    def get_output_shape_for(self, input_shapes):
        """
        Ещё 1 обязательный для lasagne (но не для Вас) метод слоя - вычисление размерности выхода по размерностям входов
        """
        observation_shape,last_memory_state_shape,controls_shape = input_shapes
        
        return last_memory_state_shape


    
    
    
    


# Струтктурная схема
![canvas](stack-rnn.png)


* А теперь вылезай из-под стола. На самом деле всё проще.
* Всё, что не обведено жЫрной чОрной линией - уже было в предыдущей схеме
* Prev stack / Next stack - предыдущее и новое состояния стэка
* StackAugmentation - слой, который вы только что реализовали
* Stack Input и Controls - просто Dense слои
* First - это операция подсматривания в верхний элемент стака (SliceLayer)

Для вашей простоты, RNN часть уже сделана по аналогии с предыдущей сетью 
* (если вам больше нравится ваша имплементация - просто скопируйте её)

In [ ]:


#observation - сюда на каждом шаге подаётся очередная буква из входной последовательности
output_shape = (None,)
observation_layer = InputLayer(output_shape,name="obs_input")


# Token embedding
n_tokens = len(alphabet)
obs_embedding = EmbeddingLayer(observation_layer,
                                              input_size=n_tokens,
                                              output_size=n_tokens,
                                              name = "input_embedding")




#число нейронов в скрытом слое RNN - чуть меньше, чтобы сумма всех весов получилась такой же, как у RNN без стэка
n_hid_1 = 64

#Сюда прицепится предыдущее состояние RNN
prev_rnn_layer = InputLayer((None,n_hid_1),name="prev_rnn_state")


# Сюда прицепится предыдущее стостояние стэка
stack_width = 3
stack_depth = 50

prev_stack_layer = InputLayer((None,stack_depth,stack_width))



# Controls
stack_controls_layer = Слой управления стаком - принимает предыдущее состояние rnn и генерит 3 выхода, которые суммируются в 1 (softmax)




# Stack input
stack_input_layer = Полносвязный слой от того же входа, размером в 3 нейрона. В качестве нелинейности что угодно (tanh например)
    
    
#новое состояние стэка  - используем вашу функцию обновления
next_stack = StackAugmentation(stack_input_layer,
                              prev_stack_layer,
                              stack_controls_layer)


#возьмём первый элемент стэка (First), чтобы использовать в RNN
stack_top = lasagne.layers.SliceLayer(next_stack,0,1)




# RNN memory


#RNN from input
rnn_frominput = DenseLayer(obs_embedding,
                           num_units=n_hid_1,
                           name= "rnn input to hidden",
                           b= None,
                           nonlinearity = None)


#RNN from hidden

rnn_fromhidden = DenseLayer(prev_rnn_layer,
                            num_units=n_hid_1,
                            name= "rnn hidden to hidden",
                            nonlinearity = None)
                            
#RNN from stack
rnn_fromstack = Обновление RNN на основании stack_top


rnn_sum = ElemwiseSumLayer([
        rnn_frominput,
        rnn_fromhidden,
        rnn_fromstack     
    ],
    name = "rnn_sum")
#между прочим это RNN с тремя входами, один из которых зависит от её предыдущего состояние,
#и лазанья такое уже не умеет

rnn = NonlinearityLayer(rnn_sum,lasagne.nonlinearities.tanh,
                        name = "rnn nonlinearity")



# В этом словаре лежат пары  {выход скрытого состояния: вход скрытого состояния с предыдущего шага}
# допишем сюда стэк
from collections import OrderedDict
memory_dict = OrderedDict([
            (rnn,prev_rnn_layer),
            (next_stack, prev_stack_layer)
    ])


#Вероятности букв

probability_layer = lasagne.layers.DenseLayer(rnn,
                                         num_units = n_tokens,
                                         nonlinearity=  lasagne.nonlinearities.softmax,
                                         name="policy_original")

#resolver - выбирает конкретную букву пропорционально вероятностям

resolver = ProbablisticResolver(probability_layer,
                                assume_normalized=True,
                                name="resolver")


#Проверка, совпадают ли формы у состояний. Должна пройти, если вы не сделали чего-то ужасного
assert tuple(lasagne.layers.get_output_shape(resolver)) == tuple(output_shape)






#Создаём рекуррентный генератор, который
# - имеет 1 вход - observation layer
# - имеет память, описанную в memory dict ( один слой RNN )
# - генерирует буквы с вероятностями, полученными на probability layer
# - принимает решение, какую букву генерить, на слое resolver layer - пропорционально вероятностям

agent = Generator(
    observation_layer,
    memory_dict,
    probability_layer,
    resolver
    )


In [ ]:
#Получим веса - проследите, чтобы там были веса всех обучаемых частей стэка
weights = lasagne.layers.get_all_params(resolver,trainable=True)
weights

In [ ]:
total_weights = int(T.sum([T.prod(w.shape) for w in weights]).eval())
print "Всего весов:", total_weights

#если вы экспериментируете с размером нейронки - удалите следующую строку
assert 5000 < total_weights <= 5500


# Copy-paste time

Ниже следует абсолютно тот же код, что и в случае с RNN 
- желающие могут проследить 
- только без комментариев и в одной массе.
- почему для этого не созданы функции - чтобы было проще отлаживать
- __от вас требуется только скопировать туда строчку с оптимизатором__

In [ ]:
probas_seq = agent.get_sessions(session_length=SEQ_LENGTH,
                             recorded_sequences=sequences_batch,
                             batch_size=batch_size,)[-1]

model_loss = lasagne.objectives.categorical_crossentropy(
    T.maximum(probas_seq[:,:-1],1e-10).reshape([-1,n_tokens]),
    sequences_batch[:,1:].ravel()
).mean()

loss = model_loss + regularize_network_params(resolver,l2)*10**-5


In [ ]:

updates = ваш любивый оптимизатор

In [ ]:
train_fun = theano.function([sequences_batch],[loss],updates=updates)
evaluation_fun = theano.function([sequences_batch],[loss,model_loss,reg_l2])

In [ ]:
gen_batch_size = T.scalar('generated batch size','int32')
generated_action_seq = agent.get_sessions(session_length=SEQ_LENGTH,
                             batch_size=gen_batch_size,)[-2]
get_sequences = theano.function([gen_batch_size],generated_action_seq)

# Цикл обучения - Stack-augmented RNN

* Всё как обычно - учимся и раз в 100 итераций считаем метрики

* Метрики нас интересуют 3 - 
 * llh - простой loglikelihood - должен падать +- шум
 * доля правильных последовательностей - с какой вероятностью rnn сгенерит подряд сколько-то a, потом b, потом c - и потом начнёт новую строку.
   * для формалистов - доля строк, которые полностью матчатся регээкспом "^|a+b+c+"
   * тоже должен в целом расти, пока не выйдёт на асимптоту близь единицы
 * ошибка в количестве C
   * рассматриваем только "правильные" последовательности по определению из предыдущего пункта
   * считаем количество букв a (n), количество букв b (m). Правильное количество букв с - n+m
   * Считаем средний модуль разности между сгенерированным количеством С и правильным.
   * Формально метрика называется mean absolute error или MAE

In [ ]:
from collections import defaultdict
metrics = defaultdict(dict)

In [ ]:

for i in range(N_EPOCHS):
    
    #породим строки
    new_batch = generate_sequence(BATCH_SIZE,SEQ_LENGTH)
    #Потренируем чудовище
    train_fun(new_batch)
    
    #Once in a while, напечатаем метрики
    if i % REPORT_RATE==0:
        
        loss_components = evaluation_fun(new_batch)
        print "iter:%i\tfull:%.5f\tllh:%.5f\treg:%.5f"%tuple([i]+map(float,loss_components))        
        
        metrics['crossentropy'][i]=float(loss_components[1])
        

        examples = get_sequences(1000)
        
        correctness_ratio,c_count_mae = get_metrics(examples,alphabet)
        

        metrics["correctness_rates"][i] = correctness_ratio
        metrics["c_count_errors"][i]=c_count_mae
        
        print "Доля последовательностей правильной формы: %.5f"%(correctness_ratio)
        print "MAE Ошибка по количеству C среди правильных: %.5f"%(c_count_mae)
        
        for tid_line in examples[:3]:
            print ' '.join(map(alphabet.__getitem__,tid_line))

    

# Тестики

In [ ]:


get_5_last = lambda metric_name: map(lambda v: v[1],sorted(metrics[metric_name].items(),key=lambda v:v[0])[-5:])


#проверяем, что минимум по последним 5 значениям метрики не хуже порога

assert min(get_5_last("crossentropy")) <= 0.25
assert min(get_5_last("c_count_errors")) <= 1
assert min(get_5_last("correctness_rates")) >= 0.9


# Кривулины

In [ ]:
for metric in metrics:
    plt.figure(figsize=[14,8])
    plt.plot(*zip(*sorted(metrics[metric].items(),key=lambda (k,v):k)),label='Stack RNN '+metric)
    plt.plot(*zip(*sorted(rnn_metrics[metric].items(),key=lambda (k,v):k)),label='Simple RNN '+metric)
    plt.legend(loc='best')
    plt.ylabel("popugai")
    plt.xlabel("epoch")
    plt.grid()
    plt.show()

# Вопросы на CPU-time

* Как изменится (и изменится ли) картина, если уменьшить/увеличить
 * количество нейронов в RNN в 2 раза больше/меньше как со стэком, так и без?
 * размер стэка (stack width) и его глубину? Какая минмальная глубина нужна, чтобы был видимый эффект?
 * сделать ещё 1+ стэк?

# Кто победил?

Ну конечно же, <так кто же победил?>, однако <комментарии>
